In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy import stats
import matplotlib as plt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn as sk
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn import tree, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report 

loading the given data

In [2]:
df = pd.read_csv('pd_speech_features.csv')
new_header = df.iloc[0] 
df = df[1:] 
df.columns = new_header 
df.head()

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
1,0,1,0.85247,0.71826,0.57227,240,239,0.00806353,8.68E-05,0.00218,...,1.562,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
2,0,1,0.76686,0.69481,0.53966,234,233,0.008258256,7.31E-05,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.178,1
3,0,1,0.85083,0.67604,0.58982,232,231,0.00833959,6.04E-05,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
4,1,0,0.41121,0.79672,0.59257,178,177,0.010857733,0.000182739,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.846,6.265,4.0603,1
5,1,0,0.3279,0.79782,0.53028,236,235,0.008161574,0.002668863,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1


checking the data types 

In [3]:
dataTypes = df.dtypes
print(dataTypes)

0
id                           object
gender                       object
PPE                          object
DFA                          object
RPDE                         object
                              ...  
tqwt_kurtosisValue_dec_33    object
tqwt_kurtosisValue_dec_34    object
tqwt_kurtosisValue_dec_35    object
tqwt_kurtosisValue_dec_36    object
class                        object
Length: 755, dtype: object


check to see if there are any missing values

In [4]:
df.isna().sum()

0
id                           0
gender                       0
PPE                          0
DFA                          0
RPDE                         0
                            ..
tqwt_kurtosisValue_dec_33    0
tqwt_kurtosisValue_dec_34    0
tqwt_kurtosisValue_dec_35    0
tqwt_kurtosisValue_dec_36    0
class                        0
Length: 755, dtype: int64

VIF or Variance Inflation Factor allows you to determine the strength of the correlation between the various independent variables. It is calculated by taking a variable and regressing it against every other variables.

    1 — features are not correlated
    1<VIF<5 — features are moderately correlated
    VIF>5 — features are highly correlated
    VIF>10 — high correlation between features and is cause for concern

In [5]:
def calculate_vif(df, features):    
    vif, tolerance = {}, {}    # all the features that you want to examine
    for feature in features:
        # extract all the other features you will regress against
        X = [f for f in features if f != feature]        
        X, y = df[X], df[feature]        # extract r-squared from the fit
        r2 = LinearRegression().fit(X, y).score(X, y)                
        
        # calculate tolerance
        tolerance[feature] = 1 - r2        # calculate VIF
        vif[feature] = 1/(tolerance[feature])    # return VIF DataFrame
    return pd.DataFrame({'VIF': vif, 'Tolerance': tolerance})

In [6]:
calculate_vif(df=df, features=df.columns[:10])

,VIF,Tolerance
id,1.040491,0.961085
gender,1.408752,0.709848
PPE,1.359056,0.735805
DFA,1.121647,0.891546
RPDE,2.015153,0.496240
numPulses,9588.221075,0.000104
numPeriodsPulses,9604.315025,0.000104
meanPeriodPulses,5.376014,0.186011
stdDevPeriodPulses,1.471749,0.679464
locPctJitter,2.403930,0.415985


In [7]:
calculate_vif(df=df, features=["gender",
                               "PPE",
                               "DFA",
                               "RPDE",
                               "numPeriodsPulses",
                               "meanPeriodPulses",
                               "stdDevPeriodPulses",
                                "locPctJitter"
                                ])

,VIF,Tolerance
gender,1.385344,0.721843
PPE,1.358595,0.736055
DFA,1.106663,0.903617
RPDE,2.005294,0.498680
numPeriodsPulses,5.692658,0.175665
meanPeriodPulses,5.235645,0.190998
stdDevPeriodPulses,1.444875,0.692102
locPctJitter,1.863464,0.536635


this wont help us here either as we have too many columns

In [8]:
X  = df.iloc[: , 1:754] #get rid of the id
y = df.iloc[: , 754:755]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, shuffle = True)

now we need to normalize the dataset using standard scaler

In [9]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

dealing with the outliers

In [10]:
y_train = y_train[(np.abs(stats.zscore(X_train)) < 3).all(axis=1)]
X_train = X_train[(np.abs(stats.zscore(X_train)) < 3).all(axis=1)]

it is asked to check and see what happens if we have an array of 1s only as prediction 

In [11]:
one = []

for i in range(len(y_test)):
    
    one.append("1")
    
accuracy_score(y_test, one)

0.7004405286343612

### Decision Tree

In [12]:
clf = tree.DecisionTreeClassifier(random_state=42, max_depth=40)
res = cross_val_score(clf, X_train, y_train).mean()
print('accuracy:', res.mean())

accuracy: 0.7166666666666667


In [13]:
rand_ls = [0, 0, 42, 42]
depth_ls = [10, 20, 20, 40]
acc_ls = [0.66, 0.66, 0.71, 0.71]

vdf_lab = {"random state": rand_ls, "max depth": depth_ls, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,random state,max depth,accuracy
0,0,10,0.66
1,0,20,0.66
2,42,20,0.71
3,42,40,0.71


In [14]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_train) #train set prediction
accuracy_score(y_train, y_predict)

1.0

In [15]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test) #test set prediction
accuracy_score(y_test, y_predict)

0.748898678414097

In [16]:
confusion_matrix(y_test, y_predict)

array([[ 38,  30],
       [ 27, 132]], dtype=int64)

### KNN

In [17]:
clf = KNeighborsClassifier(n_neighbors=2)
res = cross_val_score(clf, X_train, y_train).mean()
print('accuracy:', res.mean())

accuracy: 0.85


C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

In [18]:
nbrs = [1, 2, 10]
acc_ls = [0.81, 0.85, 0.8]

vdf_lab = {"neighbors": nbrs, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,neighbors,accuracy
0,1,0.81
1,2,0.85
2,10,0.80


In [19]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_train) #train set prediction
accuracy_score(y_train, y_predict)

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.95

In [20]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test) #test set prediction
accuracy_score(y_test, y_predict)

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.73568281938326

In [21]:
confusion_matrix(y_test, y_predict)

array([[ 46,  22],
       [ 38, 121]], dtype=int64)

### SVM

In [22]:
clf = svm.SVC(kernel='sigmoid', degree=1, C=1)
res = cross_val_score(clf, X_train, y_train).mean()
print('accuracy:', res.mean())

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


accuracy: 0.8166666666666668


C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [23]:
krnl = ["linear", "poly", "poly", "rbf", "sigmoid"]
dgree = [1, 1, 5, 1, 1]
c = [1, 1, 60, 1, 1]
acc_ls = [0.78, 0.78, 0.81, 0.81, 0.81]

vdf_lab = {"kernel": krnl, "degree": dgree, "C": c, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,kernel,degree,C,accuracy
0,linear,1,1,0.78
1,poly,1,1,0.78
2,poly,5,60,0.81
3,rbf,1,1,0.81
4,sigmoid,1,1,0.81


In [24]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_train) #train set prediction
accuracy_score(y_train, y_predict)

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8166666666666667

In [25]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test) #test set prediction
accuracy_score(y_test, y_predict)

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7224669603524229

In [26]:
confusion_matrix(y_test, y_predict)

array([[  6,  62],
       [  1, 158]], dtype=int64)

### Random Forest

In [27]:
clf = RandomForestClassifier(random_state=0, max_depth=10)
res = cross_val_score(clf, X_train, y_train).mean()
print('accuracy:', res.mean())

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column

accuracy: 0.8


In [28]:
rand_ls = [0, 0, 42, 42]
depth_ls = [10, 20, 20, 40]
acc_ls = [0.8, 0.8, 0.8, 0.8]

vdf_lab = {"random state": rand_ls, "max depth": depth_ls, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,random state,max depth,accuracy
0,0,10,0.8
1,0,20,0.8
2,42,20,0.8
3,42,40,0.8


In [29]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_train) #train set prediction
accuracy_score(y_train, y_predict)

C:\Users\Niloufar\AppData\Local\Temp/ipykernel_24532/571928701.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


1.0

In [30]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test) #test set prediction
accuracy_score(y_test, y_predict)

C:\Users\Niloufar\AppData\Local\Temp/ipykernel_24532/883292453.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


0.7577092511013216

In [31]:
confusion_matrix(y_test, y_predict)

array([[ 27,  41],
       [ 14, 145]], dtype=int64)

In [32]:
lab = ["Decision Tree", "KNN", "SVM", "Random Forest"]
acc_ls = [0.71, 0.85, 0.81, 0.8]

vdf_lab = {"model": lab, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,model,accuracy
0,Decision Tree,0.71
1,KNN,0.85
2,SVM,0.81
3,Random Forest,0.80


### PCA

In [33]:
pca = PCA(n_components=60)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_train = pd.DataFrame(X_train)
pca.fit(X_test)
X_test = pca.transform(X_test)

### Decision Tree

In [38]:
clf = tree.DecisionTreeClassifier(random_state=42, max_depth=20)
res = cross_val_score(clf, X_train, y_train).mean()
print('accuracy:', res.mean())

accuracy: 0.8666666666666666


In [39]:
rand_ls = [0, 0, 42, 42]
depth_ls = [10, 20, 20, 40]
acc_ls = [0.8, 0.8, 0.86, 0.86]

vdf_lab = {"random state": rand_ls, "max depth": depth_ls, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,random state,max depth,accuracy
0,0,10,0.80
1,0,20,0.80
2,42,20,0.86
3,42,40,0.86


In [40]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_train) #train set prediction
accuracy_score(y_train, y_predict)

1.0

In [41]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test) #test set prediction
accuracy_score(y_test, y_predict)

0.4801762114537445

In [42]:
confusion_matrix(y_test, y_predict)

array([[10, 58],
       [60, 99]], dtype=int64)

### KNN

In [46]:
clf = KNeighborsClassifier(n_neighbors=2)
res = cross_val_score(clf, X_train, y_train).mean()
print('accuracy:', res.mean())

accuracy: 0.85


C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

In [47]:
nbrs = [1, 2, 10]
acc_ls = [0.81, 0.85, 0.8]

vdf_lab = {"neighbors": nbrs, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,neighbors,accuracy
0,1,0.81
1,2,0.85
2,10,0.80


In [48]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_train) #train set prediction
accuracy_score(y_train, y_predict)

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.95

In [49]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test) #test set prediction
accuracy_score(y_test, y_predict)

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.5682819383259912

In [50]:
confusion_matrix(y_test, y_predict)

array([[ 14,  54],
       [ 44, 115]], dtype=int64)

### SVM

In [53]:
clf = svm.SVC(kernel='sigmoid', degree=1, C=1)
res = cross_val_score(clf, X_train, y_train).mean()
print('accuracy:', res.mean())

accuracy: 0.8166666666666668


C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

In [54]:
krnl = ["linear", "poly", "poly", "rbf", "sigmoid"]
dgree = [1, 1, 5, 1, 1]
c = [1, 1, 60, 1, 1]
acc_ls = [0.78, 0.78, 0.81, 0.81, 0.81]

vdf_lab = {"kernel": krnl, "degree": dgree, "C": c, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,kernel,degree,C,accuracy
0,linear,1,1,0.78
1,poly,1,1,0.78
2,poly,5,60,0.81
3,rbf,1,1,0.81
4,sigmoid,1,1,0.81


In [55]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_train) #train set prediction
accuracy_score(y_train, y_predict)

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8166666666666667

In [56]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test) #test set prediction
accuracy_score(y_test, y_predict)

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7048458149779736

In [57]:
confusion_matrix(y_test, y_predict)

array([[  4,  64],
       [  3, 156]], dtype=int64)

### Random Forest

In [61]:
clf = RandomForestClassifier(random_state=42, max_depth=40)
res = cross_val_score(clf, X_train, y_train).mean()
print('accuracy:', res.mean())

C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Niloufar\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column

accuracy: 0.8166666666666668


In [62]:
rand_ls = [0, 0, 42, 42]
depth_ls = [10, 20, 20, 40]
acc_ls = [0.81, 0.81, 0.81, 0.81]

vdf_lab = {"random state": rand_ls, "max depth": depth_ls, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,random state,max depth,accuracy
0,0,10,0.81
1,0,20,0.81
2,42,20,0.81
3,42,40,0.81


In [63]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_train) #train set prediction
accuracy_score(y_train, y_predict)

C:\Users\Niloufar\AppData\Local\Temp/ipykernel_24532/571928701.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


1.0

In [64]:
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test) #test set prediction
accuracy_score(y_test, y_predict)

C:\Users\Niloufar\AppData\Local\Temp/ipykernel_24532/883292453.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


0.6431718061674009

In [65]:
confusion_matrix(y_test, y_predict)
lab = ["Decision Tree", "KNN", "SVM", "Random Forest"]
acc_ls = [0.86, 0.85, 0.81, 0.81]

vdf_lab = {"model": lab, "accuracy": acc_ls}
vdf = pd.DataFrame(vdf_lab)
vdf

,model,accuracy
0,Decision Tree,0.86
1,KNN,0.85
2,SVM,0.81
3,Random Forest,0.81


In [66]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.22      0.07      0.11        68
           1       0.69      0.89      0.78       159

    accuracy                           0.64       227
   macro avg       0.45      0.48      0.44       227
weighted avg       0.55      0.64      0.58       227



since my first data was not appropriate and my models were over fitting so implementing PCA helped me increase my accuracy 

so far KNN had the best result both before and after PCA